In [22]:
d = {
        "date": '2025-01-01',
        "transaction_type": 'deposit',
        "count": 30,
        "sum": 124.23
    }

key = d['date']
count_column_family_records = {c[0]: c[1] for c in d.items() if c[0] != 'date' and  c[0] in(['transaction_type', 'count'])}
sum_column_family_records =  {c[0]: c[1] for c in d.items() if c[0] != 'date' and  c[0] in(['transaction_type', 'sum'])}
sum_column_family_records

{'transaction_type': 'deposit', 'sum': 124.23}

In [46]:
','.join(['2025-05-20'])

'2025-05-20'

In [29]:
dates= '2025-01-01,2025-01-03,2025-01-04' 
date_suffixes = dates.split(',')
[f"gs://{'financial_transactions_daily_summary'}/financial_transaction_daily_summary_{date}.csv" for date in date_suffixes]

['gs://financial_transactions_daily_summary/financial_transaction_daily_summary_2025-01-01.csv',
 'gs://financial_transactions_daily_summary/financial_transaction_daily_summary_2025-01-03.csv',
 'gs://financial_transactions_daily_summary/financial_transaction_daily_summary_2025-01-04.csv']

In [43]:
import apache_beam as beam
from apache_beam.options.pipeline_options import PipelineOptions, SetupOptions
from apache_beam.io.gcp.bigtableio import WriteToBigTable
from google.cloud.bigtable.row import DirectRow
import csv
import re 

from google.cloud import storage
from google.cloud import bigtable

PROJECT_ID  = "cloud-sql-big-table-data-flow"
BIGTABLE_INSTANCE_ID = "fin-bt"
BIGTABLE_TABLE_ID    = "daily_transactions_summaries_from_csv"

bigtable_client   = bigtable.Client(project=PROJECT_ID, admin=True)
bigtable_instance = bigtable_client.instance(BIGTABLE_INSTANCE_ID)
bigtable_table    = bigtable_instance.table(BIGTABLE_TABLE_ID)

def scan_all():
    print("\n--- Scan all rows ---")
    partial = bigtable_table.read_rows()
    partial.consume_all()
    for row_key, row in partial.rows.items():
        print(row_key.decode(), {
            f"{cf}:{col.decode()}": cell[0].value.decode()
            for cf, cols in row.cells.items()
            for col, cell in cols.items()
        })

scan_all()        


--- Scan all rows ---


I0000 00:00:1747725211.250246 11308486 check_gcp_environment_no_op.cc:29] ALTS: Platforms other than Linux and Windows are not supported


2000-11-05 {'count:deposit': '300', 'sum:deposit': '4000'}
2025-05-14 {'count:deposit': '356', 'count:deposit_lowTransactionValue': 'false', 'count:payment': '349', 'count:payment_lowTransactionValue': 'false', 'count:transfer': '347', 'count:transfer_lowTransactionValue': 'false', 'count:withdrawal': '348', 'count:withdrawal_lowTransactionValue': 'false', 'sum:deposit': '-39013.75', 'sum:payment': '13194.330000000002', 'sum:transfer': '-70032.12999999999', 'sum:withdrawal': '43342.41'}
2025-05-15 {'count:deposit': '711', 'count:deposit_lowTransactionValue': 'false', 'count:payment': '704', 'count:payment_lowTransactionValue': 'false', 'count:transfer': '646', 'count:transfer_lowTransactionValue': 'false', 'count:withdrawal': '649', 'count:withdrawal_lowTransactionValue': 'false', 'sum:deposit': '-3203.1699999999937', 'sum:payment': '-33785.06', 'sum:transfer': '-39880.80999999999', 'sum:withdrawal': '23924.22000000001'}
2025-05-16 {'count:deposit': '168', 'count:deposit_lowTransaction

In [37]:
def delete_row(key):
    print(f"\n--- Deleting row `{key}` ---")
    r = bigtable_table.direct_row(key)
    r.delete()      # deletes entire row
    r.commit()
    print("Deleted")

dates_to_remove = ['2025-05-17', '2025-05-18']

for d in dates_to_remove:
    delete_row(d)


--- Deleting row `2025-05-17` ---
Deleted

--- Deleting row `2025-05-18` ---
Deleted


In [36]:
import csv

def read_line(line):
    # line: a single line of CSV text
    fields = next(csv.reader([line]))
    if fields and fields[0] == 'transaction_date_yyyymmdd':  # adjust if actual header is different
        return None
    else:
        return {
            "date": fields[0],
            "transaction_type": fields[1],
            "count": int(fields[2]),
            "sum": float(fields[3])
        }

file_path = '/Users/delgadonoriega/Desktop/gcp-data-eng-bootcamp/Module_2_class_1/financial-transactions-cloud-sql/flex_pipeline-dataflow/financial_transaction_daily_summary_2025-05-14.csv'

results = []

try:
    with open(file_path, 'r') as csvfile:
        for line in csvfile:
            processed_data = read_line(line.strip())
            print(processed_data)
            if processed_data:
                results.append(processed_data)
except FileNotFoundError:
    print(f"❌ Error: File not found at {file_path}")


None
{'date': '2025-05-14', 'transaction_type': 'deposit', 'count': 356, 'sum': -39013.75}
{'date': '2025-05-14', 'transaction_type': 'payment', 'count': 349, 'sum': 13194.330000000002}
{'date': '2025-05-14', 'transaction_type': 'transfer', 'count': 347, 'sum': -70032.12999999999}
{'date': '2025-05-14', 'transaction_type': 'withdrawal', 'count': 348, 'sum': 43342.41}


In [ ]:
# files_not_yet_processed_default_parameter(generate_dates_captured_in_GCS(),
#                                           fetch_captured_dates_in_big_table())

# def insert_demo_rows():
#     rows = [
#         ("2000-11-05", {"count:deposit": "300", "sum:deposit": "4000"})
#     ]
#     for key, data in rows:
#         r = bigtable_table.direct_row(key)
#         for colfam_col, val in data.items():
#             cf, col = colfam_col.split(":", 1)
#             r.set_cell(cf, col, val)
#         r.commit()
#     print("Inserted demo rows")

# def scan_all():
#     print("\n--- Scan all rows ---")
#     partial = bigtable_table.read_rows()
#     partial.consume_all()
#     for row_key, row in partial.rows.items():
#         print(row_key.decode(), {
#             f"{cf}:{col.decode()}": cell[0].value.decode()
#             for cf, cols in row.cells.items()
#             for col, cell in cols.items()
#         })


# scan_all()

In [ ]:
# else:
#     key = rec['date'].encode("utf-8")
#     row = DirectRow(row_key=key)
#     # separate 2 dicts for the different column families
#     count_column_family_records = {c[0]: c[1] for c in rec.items() if c[0] != 'date' and  c[0] in(['transaction_type', 'count'])}
#     sum_column_family_records =  {c[0]: c[1] for c in rec.items() if c[0] != 'date' and  c[0] in(['transaction_type', 'sum'])}
    
#     #Insert Count:x values
#     for k,v in count_column_family_records.items():
#         row.set_cell("count", k, str(v).encode("utf-8"))
#     #transformation: flag low number of transaction 
#         if rec["count"] < 30:
#             row.set_cell("count", f"{rec['transaction_type']}_lowTransactionValue", b"true")
#         else:
#             row.set_cell("count", f"{rec['transaction_type']}_lowTransactionValue", b"False")
#     #Insert Count:x values
#     for k,v in sum_column_family_records.items():
#         row.set_cell("sum", k, str(v).encode("utf-8"))

#     return row